Recurrent Neural Network from Scratch

In [5]:
import numpy as np

Utility functions

In [6]:

def softmax(x):
    # Shift indices to avoid overflow
    e_x = np.exp(x - np.max(x))
    return e_x / np.sum(e_x)


Main RNN class:
- uses BPTT using AdaGrad instead of SGD
- target text generation using RNN
- simple RNN model fully implemented in numpy

In [ ]:
class RNN:
    def __init__(self, vocab_size, hidden_size, seq_length, learning_rate=0.1):
        """
        vocab_size: Number of unique characters (input/output dimension)
        hidden_size: Number of neurons in the hidden layer
        seq_length: Number of time steps to unroll for backprop
        """
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        self.lr = learning_rate

        # initialization: simple random * 0.01 works for simple RNNs
        
        # W_xh: Input to Hidden
        self.Wxh = np.random.randn(hidden_size, vocab_size) * 0.01
        
        # W_hh: Hidden to Hidden (Recurrent connection)
        self.Whh = np.random.randn(hidden_size, hidden_size) * 0.01
        
        # W_hy: Hidden to Output
        self.Why = np.random.randn(vocab_size, hidden_size) * 0.01
        
        # Biases
        self.bh = np.zeros((hidden_size, 1)) # Hidden bias
        self.by = np.zeros((vocab_size, 1))  # Output bias

        # Memory for Adagrad optimization (makes convergence much faster/stable)
        # m variables = memory variables used for the Adagrad update
        self.mWxh, self.mWhh, self.mWhy = np.zeros_like(self.Wxh), np.zeros_like(self.Whh), np.zeros_like(self.Why)
        self.mbh, self.mby = np.zeros_like(self.bh), np.zeros_like(self.by)

    def forward(self, inputs, h_prev):
        """
        inputs: List of integers (indices of characters) for the sequence
        h_prev: The hidden state from the previous time step (t-1)
        """

        # storage for future use in backprop
        xs, hs, ys, ps = {}, {}, {}, {}
        hs[-1] = np.copy(h_prev) # Store previous state at t=-1
        
        loss = 0
        
        # Loop through time steps
        for t in range(len(inputs)):
            # One-Hot Encode Input
            xs[t] = np.zeros((self.vocab_size, 1))
            xs[t][inputs[t]] = 1 # puts 1 at the index of the input character for eg if input = 3, then xs[t] = [0,0,0,1,0,...]
            
            # Update Hidden State
            # h_t = tanh(Wxh * x_t + Whh * h_{t-1} + bias)
            hs[t] = np.tanh(np.dot(self.Wxh, xs[t]) + np.dot(self.Whh, hs[t-1]) + self.bh)
            
            # y_t = Why * h_t + bias
            ys[t] = np.dot(self.Why, hs[t]) + self.by
            
            #  Probabilities (Softmax)
            ps[t] = softmax(ys[t])
            
        return xs, hs, ys, ps

    def backward(self, inputs, targets, xs, hs, ps):
        """
        Performs Backpropagation Through Time (BPTT)
        """
        # Initialize gradients to zero
        dWxh, dWhh, dWhy = np.zeros_like(self.Wxh), np.zeros_like(self.Whh), np.zeros_like(self.Why)
        dbh, dby = np.zeros_like(self.bh), np.zeros_like(self.by)
        
        # Gradient of hidden state from the "future" (starts at 0)
        dhnext = np.zeros_like(hs[0])
        
        # Loop BACKWARDS through time
        for t in reversed(range(len(inputs))):
            # Output Gradient (Cross Entropy + Softmax)
            # dy = probs - 1 (at target index)
            dy = np.copy(ps[t])
            dy[targets[t]] -= 1 
            
            # Gradient for W_hy and b_y
            dWhy += np.dot(dy, hs[t].T)
            dby += dy
            
            # Gradient for Hidden State
            # Comes from two sources:
            # a) The output of this step (dy)
            # b) The hidden state of the next step (dhnext)
            dh = np.dot(self.Why.T, dy) + dhnext
            
            # Gradient through Tanh nonlinearity
            # dtanh = (1 - h^2) * dh
            dhraw = (1 - hs[t] * hs[t]) * dh
            
            # Gradients for W_xh, W_hh, b_h
            dbh += dhraw
            dWxh += np.dot(dhraw, xs[t].T)
            dWhh += np.dot(dhraw, hs[t-1].T)
            
            # Pass gradient to the previous time step
            dhnext = np.dot(self.Whh.T, dhraw)
            
        # Clip gradients to mitigate exploding gradients problem
        for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
            np.clip(dparam, -5, 5, out=dparam)
            
        return dWxh, dWhh, dWhy, dbh, dby

    def update_params(self, dWxh, dWhh, dWhy, dbh, dby):
        """
        Updates weights using Adagrad (Adaptive Gradient).
        Standard SGD is very unstable for RNNs; Adagrad is much better for convergence.
        """
        for param, dparam, mem in zip([self.Wxh, self.Whh, self.Why, self.bh, self.by],
                                      [dWxh, dWhh, dWhy, dbh, dby],
                                      [self.mWxh, self.mWhh, self.mWhy, self.mbh, self.mby]):
            mem += dparam * dparam
            param += -self.lr * dparam / np.sqrt(mem + 1e-8)

    def sample(self, seed_ix, n):
        """
        Generates a sequence of integers from the model.
        """
        h = np.zeros((self.hidden_size, 1)) # Reset hidden state
        x = np.zeros((self.vocab_size, 1))
        x[seed_ix] = 1 # One-hot input
        
        ixes = []
        
        for t in range(n):
            # Forward pass (one step)
            h = np.tanh(np.dot(self.Wxh, x) + np.dot(self.Whh, h) + self.bh)
            y = np.dot(self.Why, h) + self.by
            p = softmax(y)
            
            # Sample from the distribution
            ix = np.random.choice(range(self.vocab_size), p=p.ravel())
            
            # Prepare input for next step
            x = np.zeros((self.vocab_size, 1))
            x[ix] = 1
            ixes.append(ix)
    
        # returns the list character indices generated from RNN        
        return ixes

    def train(self, data, epochs=1000):
        # Preprocessing (Char <-> Int)
        chars = list(set(data))
        data_size, vocab_size = len(data), len(chars)
        char_to_ix = { ch:i for i,ch in enumerate(chars) } # return char for index
        ix_to_char = { i:ch for i,ch in enumerate(chars) } # return index for char
        
        print(f"Data size: {data_size}, Vocab size: {vocab_size}")
        
        # Reset parameters for this specific vocab size
        self.__init__(vocab_size, self.hidden_size, self.seq_length, self.lr)
        
        n, p = 0, 0 # Iteration counter, data pointer
        h_prev = np.zeros((self.hidden_size, 1)) # Initial hidden state
        
        for i in range(epochs):
            # Prepare inputs and targets
            # If we are at end of text, reset pointer
            if p + self.seq_length + 1 >= len(data): 
                h_prev = np.zeros((self.hidden_size, 1))
                p = 0
                
            inputs = [char_to_ix[ch] for ch in data[p:p+self.seq_length]]
            targets = [char_to_ix[ch] for ch in data[p+1:p+self.seq_length+1]]

            # Forward
            xs, hs, ys, ps = self.forward(inputs, h_prev)
            
            # Calculate Loss
            loss = 0
            for t in range(len(inputs)):
                loss += -np.log(ps[t][targets[t], 0])
            
            # Backward
            dWxh, dWhh, dWhy, dbh, dby = self.backward(inputs, targets, xs, hs, ps)
            
            # Update
            self.update_params(dWxh, dWhh, dWhy, dbh, dby)
            
            # Update pointers
            h_prev = hs[len(inputs)-1] # Carry over hidden state
            p += self.seq_length
            
            if i % 1000 == 0:
                print(f'Epoch {i}, Loss: {loss:.4f}')
                # Sample text to see progress
                sample_ix = self.sample(inputs[0], 150)
                txt = ''.join(ix_to_char[ix] for ix in sample_ix)
                print(f"----\n{txt}\n----")

In [8]:
# Prepare the Data ----------------------
# The first verse poem Hope is the thing with feathers by Emily Dickinson
text_segment = "Hope is the thing with feathers that perches in the soul. And sings the tune without the words and never stops at all. "

# We repeat the text to create a longer stream for the training loop
# This simulates a dataset where this sentence structure is common.
data = text_segment * 100 

print(f"Total characters in dataset: {len(data)}")
print(f"Unique characters: {len(set(data))}")

# Configure and Train ----------------------------

# Parameters:
# hidden_size=64: Enough memory to store the sentence structure
# seq_length=25: The RNN looks at 25 characters at a time to predict the 26th
# learning_rate=0.1: Standard for character-level RNNs
rnn = RNN(vocab_size=0, hidden_size=64, seq_length=25, learning_rate=0.1)

print("\n--- Starting Training ---")
print(f"Target pattern: {text_segment}\n")

# We train for 10000 epochs to ensure it perfectly memorizes the phrase
rnn.train(data, epochs=10000)

# ==========================================
# Final Verification (Generation)
# ==========================================
print("\n--- Final Generation ---")
# Seed the model with the first character 'H' and ask it to generate 200 characters
chars = list(set(data))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }


seed_char = 'H'
seed_idx = char_to_ix[seed_char]

sample_ix = rnn.sample(seed_idx, n=200)
generated_text = seed_char + ''.join(ix_to_char[ix] for ix in sample_ix)

print(generated_text)

Total characters in dataset: 11900
Unique characters: 22

--- Starting Training ---
Target pattern: Hope is the thing with feathers that perches in the soul. And sings the tune without the words and never stops at all. 

Data size: 11900, Vocab size: 22
Epoch 0, Loss: 77.2765
----
ucgw fwlncupt.ieAuHvuiovho.acHrggacw.HrcgpatfuAuAAtuhtcHHAt..Hf.oun tdHcHfw iifAndiueiHih.A.otuwigdrHtsHwApHHcwHAhAipAicrAdivferHA Hlfshoatfrfevucfd e
----
Epoch 1000, Loss: 1.1133
----
 thang with feathers that perches in the wotun nd ithout the words and never stops at pll. oll. ings in al. ings without the tope without nnd is the t
----
Epoch 2000, Loss: 0.3242
----
pe at all. Hope is the thing with feathers th thope ws thercuns in the soul. And sings the tune without the words and never stops at all. Hope is the 
----
Epoch 3000, Loss: 0.1763
----
thand in the sings the tune without the words and never stops at all. Hope is the thing with feathers that perches in the soul. And sings the tune wit
----
Epoc